In [1]:
import os
import pandas as pd
import numpy as np
import torch
from src import train, predict
from src.utils import plot_fit_figures
import traceback
from src.lstm import LSTM
from sklearn.model_selection import ParameterGrid
import itertools
from src.utils import eval_func
from tqdm import tqdm

In [2]:
# 超参数的可能值
best_params = {
    'hidden_size': 56,
    'num_layers': 2,
    'batch_size': 64,
    'seq_len': 64,
}

hidden_size_options = [64]
num_layers_options = [2]
batch_size_options = [64]
seq_len_options = [64]
# 所有超参数组合
param_grid = list(itertools.product(hidden_size_options, num_layers_options, batch_size_options, seq_len_options))

In [3]:
def run(best_config: dict, fund_code: str):
    model = LSTM(input_size=7,
                 output_size=7,
                 hidden_size=best_config['hidden_size'],
                 num_layers=best_config['num_layers'],
                 )
    config = {
        'model': model,
        'fund_code': fund_code,
        'data_set_length': 1000,
        'batch_size': best_config['batch_size'],
        'num_epochs': 60,
        'seq_len': best_config['seq_len'],
    }
    train(**config)
    config.pop('num_epochs')
    config.pop('data_set_length')
    predictions, groundtruths = predict(**config,predict_steps=1000)
    plot_fit_figures(fund_code=fund_code, predictions=predictions, groundtruths=groundtruths)
    mse, mae, rmse, smape = eval_func(predictions, groundtruths)
    
    print(f'mse: {mse}, mae: {mae}, rmse: {rmse}, smape: {smape}')
    return predictions, groundtruths


In [4]:
 run(best_params, fund_code='519606')


-------数据集加载中-------
Dataset type: train, data length: 717
Dataset type: valid, data length: 204
--------加载完成---------
----开始在cpu上训练------


训练中: 519606: 100%|██████████| 60/60 [00:16<00:00,  3.59it/s]


--------训练完成---------
-----loss曲线绘制完成-----
-------模型保存完成-------
Dataset type: test, data length: 409
---------开始预测---------


继续预测: 100%|██████████| 1000/1000 [00:00<00:00, 1093.06it/s]


---------预测完成---------


ValueError: operands could not be broadcast together with shapes (1320,7) (320,7) 

In [ ]:
def grid_search(param_grid, fund_code):
    with open('grid_search_results.csv', 'w', encoding="utf-8") as f:
        f.write('hidden_size,num_layers,batch_size,seq_len,mse,mae,rmse,smape\n')
        for param in tqdm(param_grid):
            hidden_size, num_layers, batch_size, seq_len = param
            model = LSTM(input_size=7,
                         output_size=7,
                         hidden_size=hidden_size,
                         num_layers=num_layers,
                         )
            config = {
                'model': model,
                'fund_code': fund_code,
                'data_set_length': 1000,
                'batch_size': batch_size,
                'num_epochs': 60,
                'seq_len': seq_len,
            }
            try:
                train(**config)
                config.pop('num_epochs')
                config.pop('data_set_length')
                predictions, groundtruths = predict(**config)
                plot_fit_figures(fund_code=fund_code, predictions=predictions, groundtruths=groundtruths)
                mse, mae, rmse, smape = eval_func(predictions, groundtruths)
                f.write(f'{hidden_size},{num_layers},{batch_size},{seq_len},{mse},{mae},{rmse},{smape}\n')

            except Exception as e:
                print(param)
                print(traceback.format_exc())
                continue
